In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso #importa la regresión lineal, la regresión ridge y la regresión lasso
from sklearn.metrics import mean_squared_error #Para calcular el error cuadrático medio
from sklearn.preprocessing import StandardScaler, PolynomialFeatures #Para estandarizar variables y obtener términos polinomiales
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, RandomizedSearchCV
import matplotlib.pyplot as plt #Para algunos gráficos

In [2]:
Empleados=pd.read_csv("https://raw.githubusercontent.com/AlcidesOxa/datos/master/ejemplo.csv",sep=';',encoding="ISO-8859-1")
Empleados.head()

,Nombres,Apellidos,Sexo,Edad,Años estudio,Ingresos,Gastos,Estado Civil,Satisfacción Trabajo,Fecha contrato,Faltas,Permisos
0,Pablo Enrique,Barrientos Gómez,Hombre,53.0,5.0,3662.0,2955.0,Divorciado(a),Regular,6/5/2010,1.0,NaN
1,Teresa Marina,Bracamonte Palma,Mujer,NaN,23.0,NaN,NaN,Soltero(a),NaN,NaN,NaN,2.0
2,Vicente,Cardoso Zerda,Hombre,39.0,7.0,4063.0,4180.0,Casado(a),Mal,18/4/2009,NaN,NaN
3,Juan José,Ceballos,Hombre,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Raúl Sergio,Claros Gonzáles,Hombre,55.0,20.0,15479.0,9344.0,Divorciado(a),Bien,12/8/2017,NaN,1.0


In [3]:
cambio={'Años estudio':'Anios_estudio', 'Estado Civil':'Estado_civil', 'Satisfacción Trabajo':'Satisfaccion_trabajo', 'Fecha contrato':'Fecha_contrato'}
Empleados.rename(columns=cambio,inplace=True)
casosborrar=[3,5,8]
Empleados=Empleados.drop(casosborrar)
varborrar=['Faltas','Permisos']
Empleados=Empleados.drop(varborrar,axis=1)
varcategoricas=['Sexo','Estado_civil','Satisfaccion_trabajo']
Empleados[varcategoricas]=Empleados[varcategoricas].astype('category')
Empleados['Fecha_contrato']=pd.to_datetime(Empleados.Fecha_contrato)
Empleados['Experiencia_lab']=Empleados.Edad-Empleados.Anios_estudio-5
Empleados['Balance']=Empleados.Ingresos-Empleados.Gastos

# Evaluación del modelo

## Variables del modelo

In [5]:
# Lista de las variables del modelo (dependiente e independientes)
vars_mod = ['Ingresos', 'Edad', 'Sexo', 'Anios_estudio', 'Estado_civil']
# Lista de variables independientes categóricas
vars_cat_mod = ['Sexo', 'Estado_civil']
# Lista de variables independientes cuantitativas
vars_cuant_mod = ['Edad', 'Anios_estudio']
# Elimina los datos perdidos del conjunto de variables del modelo
# Datos es un dataframe que sólo contiene variables del modelo y sin datos perdidos
Datos = Empleados[vars_mod].dropna()
Datos.head()

,Ingresos,Edad,Sexo,Anios_estudio,Estado_civil
0,3662.0,53.0,Hombre,5.0,Divorciado(a)
2,4063.0,39.0,Hombre,7.0,Casado(a)
4,15479.0,55.0,Hombre,20.0,Divorciado(a)
6,7553.0,30.0,Mujer,18.0,Soltero(a)
7,11612.0,33.0,Mujer,23.0,Casado(a)


In [9]:
# Obtiene la matriz X de variables cuantitativas independientes
X = Datos[vars_cuant_mod]
# Obtiene el vector Y, de la variable dependiente
Y = Datos.Ingresos
#dimensión de X
np.shape(X)

(24, 2)

## Conjuntos de entrenamiento y de prueba

In [ ]:
#Divide el conjunto de datos en datos de entrenamiento y de prueba
Xen, Xpr, Yen, Ypr = train_test_split(X, Y)
np.shape(Xen)
np.shape(Xpr)
#Para otros tamaños del conjunto de prueba, por ej 20%
Xen, Xpr, Yen, Ypr = train_test_split(X, Y, test_size=0.4)

## Evaluación dentro y fuera de la muestra de entrenamiento

In [20]:
reg = LinearRegression()
reg.fit(Xen, Yen) #Ajuste del modelo lineal con los datos de entrenamiento
Yen_p = reg.predict(Xen) #Predicción con datos de entrenamiento
Ypr_p = reg.predict(Xpr) #Predicción con datos de prueba
#imprime los errores cuadráticos medios de entrenamiento y de prueba
print('MSE entrenamiento:', mean_squared_error(Yen_p, Yen), 'MSE prueba', mean_squared_error(Ypr_p, Ypr))

MSE entrenamiento: 375791.3757633422 MSE prueba 1620896.935102719


# Validación cruzada

Es una técnica que consiste en dividir los datos de la muestra en $K$ conjuntos de datos. Luego utilizar $K-1$ de estos conjuntos para entrenar el modelo de predicción, y seleccionar el restante para probar la efectividad de la predicción. Esto se realiza repetidamente con cada uno de los conjuntos de datos en el papel de prueba, y el resto como entrenamiento. Se guardan los resultados de cada iteración, luego se obtiene un indicador resumen de éstos resultados, generalmente la media para evaluar la predicción que realiza el modelo en general.
Éste método se denomina $K$-fold cross validation. Un ejemplo de $K=5$ es:

![valcruz](https://static.packt-cdn.com/products/9781789617740/graphics/b04c27c5-7e3f-428a-9aa6-bb3ebcd3584c.png)

In [27]:
# Crea la instancia reg de Regresión Lineal
reg = LinearRegression()
# aplica la validación cruzada al modelo de regresión lineal "reg" y utiliza como criterio
# el negativo del error cuadrático medio
mses = cross_val_score(reg, X, Y, scoring='neg_mean_squared_error', cv=5)
mses = -mses
print(mses)
print(np.mean(mses))

[1733606.9538366   530793.77067227  192672.27518642  860107.1291615
 1079741.9273907 ]
879384.4112494988


# Métodos de contracción

Los métodos de contracción aumentan el sesgo de los estimadores, pero por el contrario disminuyen sus varianzas. En general, en un modelo de regresión lineal, se da el caso de que estimadores sesgados pero con poca varianza brindan mejores predicciones que modelos insesgados. Dos métodos de contracción (shrinkage) muy utilizados son la regresión Ridge y la regresión Lasso.

## Regresión Ridge

Para el modelo de regresión lineal $Y=X\beta$, la estimación ridge consiste en minimizar la función de pérdida de mínimos cuadrados ordinarios sujeto a una restricción, es decir, minimizar la suma de cuadrados de los residuales $\left( Y-X\beta \right) ^{\prime }\left( Y-X\beta \right)$ sujeto a que $\beta^{\prime }\beta=c$. La función lagrangeana es
\begin{equation*}
L=\left( Y-X\beta \right) ^{\prime }\left( Y-X\beta \right) +\lambda \beta^{\prime }\beta
\end{equation*}
resolviendo éste problema de optimización se obtiene que el estimador es
\begin{equation*}
\widehat{\beta }=\left( X^{\prime }X+\lambda I\right) ^{-1}X^{\prime}Y
\end{equation*}
Es importante apuntar que la matriz $X$ en este caso, sólo incluye las variables explicativas, no la constante.

In [30]:
#Conjunto de los valores de lambda del 1 al 99
lambdas = {'alpha': np.arange(1, 100)}
#Búsqueda del mejor valor del hiperparámetro lambda con validación cruzada de 5 grupos de prueba
ridges = GridSearchCV(Ridge(), lambdas, scoring='neg_mean_squared_error', cv=5)
#Ajuste con el mejor lambda hallado, el que hace mejor predicción
ridges.fit(X, Y)
print(ridges.best_params_) #visualiza el mejor lambda
print(ridges.best_score_) #visualiza el mejor -MSE

{'alpha': 9}
-876211.59238277


## Regresión Lasso

Para el modelo de regresión lineal $Y=X\beta$, la estimación Lasso se trata de minimizar la función de pérdida de la suma de cuadrados de los residuales $\left( Y-X\beta \right) ^{\prime }\left( Y-X\beta \right)$ sujeto a que $\sum_{j=2}^{k}\left\vert \beta _{j}\right\vert=c$. La función lagrangeana es
\begin{equation*}
L=\left( Y-X\beta \right) ^{\prime }\left( Y-X\beta \right) +\lambda
\sum_{j=2}^{k}\left\vert \beta _{j}\right\vert
\end{equation*}
Éste es un problema de programación cuadrática y se resuelve como tal.

In [31]:
alfas = {'alpha': np.arange(1, 100)}
lasos = GridSearchCV(Lasso(), alfas, scoring='neg_mean_squared_error', cv=5)
lasos.fit(X, Y)
print(lasos.best_params_)
print(lasos.best_score_)

{'alpha': 99}
-876177.2388942052


# Comparando varios modelos

## Función comparativa

In [32]:
def modelos(X,Y): #Función denominada "modelos" que recibe parámetros X y Y
  reg = LinearRegression()
  mses = cross_val_score(reg, X, Y, scoring='neg_mean_squared_error', cv=5)
  lambdas = {'alpha': np.arange(1, 100)}
  rid = Ridge(normalize = True) #Establece que se estandaricen las variables del modelo
  ridges = GridSearchCV(rid, lambdas, scoring='neg_mean_squared_error', cv=5)
  ridges.fit(X, Y)
  las = Lasso(normalize = True)
  lasos = GridSearchCV(las, lambdas, scoring='neg_mean_squared_error', cv=5,)
  lasos.fit(X, Y)
  print('reg lineal: ', np.mean(mses))
  print('hiperparam ridge: ', ridges.best_params_)
  print('reg ridge: ',ridges.best_score_)
  print('hiperparam lasso: ', lasos.best_params_)
  print('reg lasso: ',lasos.best_score_)

## Añadiendo variables dicotómicas

In [36]:
# Crea un dataframe de variables dicotómicas denominado Dicos, y elimina la primera
# categoría de cada variable categórica
Dicos = pd.get_dummies(Datos[vars_cat_mod], drop_first = True)
# Concatena los dataframes Dicos y Datos (solo variables cuantitativas independientes)
X = pd.concat([Dicos, Datos[vars_cuant_mod]], axis=1) #axis=1 es por columnas
Y = Datos.Ingresos
modelos(X,Y) #llama a la función modelos y pasa los X y Y previamente construidos

reg lineal:  -848001.4760779161
hiperparam ridge:  {'alpha': 1}
reg ridge:  -3865059.8536674296
hiperparam lasso:  {'alpha': 31}
reg lasso:  -717964.8310868747


## Añadiendo variables polinomiales

In [37]:
Y = Datos.Ingresos
for i in range(1,4):
  Xi = PolynomialFeatures(i).fit_transform(Datos[vars_cuant_mod]) #var cuant de grado i
  print('Modelo polinomial para grado', i)
  modelos(Xi, Y)

Modelo polinomial para grado 1
reg lineal:  -879384.4112494976
hiperparam ridge:  {'alpha': 1}
reg ridge:  -3845731.097956897
hiperparam lasso:  {'alpha': 17}
reg lasso:  -866530.7377703845
Modelo polinomial para grado 2
reg lineal:  -185358.96359831508
hiperparam ridge:  {'alpha': 1}
reg ridge:  -1374121.1459773984
hiperparam lasso:  {'alpha': 1}
reg lasso:  -294802.4428324547
Modelo polinomial para grado 3
reg lineal:  -3498.78362468262
hiperparam ridge:  {'alpha': 1}
reg ridge:  -853914.5332851037
hiperparam lasso:  {'alpha': 1}
reg lasso:  -87040.09486964566


## Combinando dicotómicas y términos polinomiales

In [38]:
Dicos = pd.get_dummies(Datos[vars_cat_mod], drop_first = True)
Y = Datos.Ingresos
for i in range(1,4):
  Xi = PolynomialFeatures(i).fit_transform(Datos[vars_cuant_mod]) #var cuant de grado i
  Xi = pd.DataFrame(Xi, index = Datos.index.values) #Transforma a X en un dataframe de pandas con index
  Xi = pd.concat([Dicos, Xi], axis=1) #concatena por columnas Dicos y X
  print('Modelo con dicotómicas y polinomial de grado', i)
  modelos(Xi, Y)

Modelo con dicotómicas y polinomial de grado 1
reg lineal:  -848001.4760779152
hiperparam ridge:  {'alpha': 1}
reg ridge:  -3865059.8536674296
hiperparam lasso:  {'alpha': 31}
reg lasso:  -717964.8310868746
Modelo con dicotómicas y polinomial de grado 2
reg lineal:  -227600.45467999502
hiperparam ridge:  {'alpha': 1}
reg ridge:  -1300624.500467915
hiperparam lasso:  {'alpha': 19}
reg lasso:  -353191.75845142605
Modelo con dicotómicas y polinomial de grado 3
reg lineal:  -5012.471228930982
hiperparam ridge:  {'alpha': 1}
reg ridge:  -677846.9781649212
hiperparam lasso:  {'alpha': 1}
reg lasso:  -194710.1222520163
